In [ ]:
!nvidia-smi

In [ ]:
%cd /content/drive/MyDrive/handson_srm/s1

# Import packages

In [ ]:
import numpy as np
import cv2
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from google.colab.patches import cv2_imshow

# Preprocess inputs

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

ToTensor()
 - converts the image with a pixel range of [0, 255] to a PyTorch FloatTensor of shape (C, H, W) with a range [0.0, 1.0]

Normalize() 
- output[channel] = (input[channel] - mean[channel]) / std[channel]

- Normalization helps get data within a range and reduces the skewness which helps learn faster and better. 
- Normalization can also tackle the diminishing and exploding gradients problems.

# Create Dataloaders

- Combines a dataset and a sampler
- Provides an iterable over the given dataset
- The DataLoader supports both map-style and iterable-style datasets - - - Supports with single- or multi-process loading
- Customizing loading order
- Automatic batching

In [ ]:
trainset = datasets.MNIST('trainset', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)

In [ ]:
valset = datasets.MNIST('testset', download=True, train=False, transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=True)

In [ ]:
dataiter = iter(trainloader)

Now that we have an iterator, let's visualize our data

In [ ]:
images, labels = dataiter.next()

In [ ]:
print(images.shape)
print(labels.shape)

In [ ]:
plt.imshow(images[2].numpy().squeeze(), cmap='gray_r')

In [ ]:
figure = plt.figure()
total_samples = 8
for index in range(1,total_samples+1):
    plt.subplot(2, 4, index)
    plt.axis('off')
    plt.imshow(images[index-1].numpy().squeeze(), cmap='gray_r')

In [ ]:
labels

In [ ]:
onehot_labels = nn.functional.one_hot(labels, num_classes=10)

In [ ]:
onehot_labels

# Let's build our DNN

In [ ]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))

In [ ]:
print(model)

# Untrained Inference

Lets flatten the images to a 1D array/tensor

In [ ]:
model_inputs = images.view(images.shape[0], -1)
model_inputs.shape

In [ ]:
pred_probs = model(model_inputs) # probabilities
pred_probs.shape

In [ ]:
pred_probs

In [ ]:
pred_probs.sum(dim=-1)

In [ ]:
plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')

In [ ]:
labels[0]

In [ ]:
onehot_labels[0]

In [ ]:
pred_probs[0]

In [ ]:
torch.argmax(pred_probs[0], dim=-1)

In [ ]:
labels

In [ ]:
torch.argmax(pred_probs, dim=0)

In [ ]:
criterion = nn.CrossEntropyLoss()
loss = criterion(pred_probs, labels) #calculate the NLL loss

In [ ]:
loss

In [ ]:
print('Before backward pass: \n', model[0].weight.grad)
loss.backward()
print('After backward pass: \n', model[0].weight.grad)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time_init = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time_init)/60)

In [ ]:
images, labels = next(iter(valloader))

with torch.no_grad():
    pred_probs = model(images.view(images.shape[0], -1))

torch.argmax(pred_probs, dim=-1)

In [ ]:
pred_probs

In [ ]:
print("Predicted Digit =", torch.argmax(pred_probs[0], dim=-1))

In [ ]:
labels

In [ ]:
# correct_count, all_count = 0, 0
# for images,labels in valloader:
#   for i in range(len(labels)):
#     img = images[i].view(1, 784)
#     with torch.no_grad():
#         logps = model(img)

    
#     ps = torch.exp(logps)
#     probab = list(ps.numpy()[0])
#     pred_label = probab.index(max(probab))
#     true_label = labels.numpy()[i]
#     if(true_label == pred_label):
#       correct_count += 1
#     all_count += 1

# print("Number Of Images Tested =", all_count)
# print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
torch.save(model, './my_mnist_model.pt') 